In [47]:
import pandas as pd

In [48]:
df = pd.read_csv("dataset/synthetic_logs.csv")

In [49]:
df.head()

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert


In [50]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [51]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [52]:
df.complexity.unique()

array(['bert', 'regex', 'llm'], dtype=object)

In [53]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

In [54]:
# loading pre trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [55]:
# generate embeddings for log messages
embeddings = model.encode(df['log_message'].tolist())

In [56]:
# performing DB scan
dbscan = DBSCAN(eps=0.2,min_samples=1,metric='cosine')
clusters = dbscan.fit_predict(embeddings)

In [57]:
df['cluster'] = clusters

In [58]:
df.head()

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0


In [59]:
df[df.cluster==97].head()

,timestamp,source,log_message,target_label,complexity,cluster
1114,5/12/2025 5:23,BillingSystem,Detected a data transfer attempt with insuffic...,Security Alert,bert,97
1175,4/21/2025 2:17,AnalyticsEngine,Unapproved data transfer attempt was detected,Security Alert,bert,97
1740,10/7/2025 8:13,ModernHR,Unauthenticated data transfer attempt was dete...,Security Alert,bert,97
1979,9/28/2025 14:51,BillingSystem,Detected an unauthorized attempt to transfer data,Security Alert,bert,97


In [60]:
cluster_counts = df['cluster'].unique()
cluster_counts

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135])

In [61]:
cluster_counts = df['cluster'].value_counts()
cluster_counts

cluster
0      1017
5       147
11      100
13       86
7        60
       ... 
131       1
132       1
133       1
134       1
135       1
Name: count, Length: 136, dtype: int64

In [62]:
large_clusters = cluster_counts[cluster_counts > 10].index
large_clusters

Index([ 0,  5, 11, 13,  7,  8, 21,  3,  4, 17,  6, 32, 16, 20,  9,  1, 10, 34,
       53, 14, 52, 18, 42, 25, 59, 26],
      dtype='int64', name='cluster')

In [63]:
for cluster in large_clusters:
    print(f"Cluster {cluster}:") 
    print(df[df['cluster'] == cluster]['log_message'].head(5).to_string(index=False))
    print()

Cluster 0:
nova.osapi_compute.wsgi.server [req-b9718cd8-f6...
nova.osapi_compute.wsgi.server [req-4895c258-b2...
nova.osapi_compute.wsgi.server [req-ee8bc8ba-92...
nova.osapi_compute.wsgi.server [req-f0bffbc3-5a...
nova.osapi_compute.wsgi.server [req-2bf7cfee-a2...

Cluster 5:
nova.compute.claims [req-a07ac654-8e81-416d-bfb...
nova.compute.claims [req-d6986b54-3735-4a42-907...
nova.compute.claims [req-72b4858f-049e-49e1-b31...
nova.compute.claims [req-5c8f52bd-8e3c-41f0-95a...
nova.compute.claims [req-d38f479d-9bb9-4276-968...

Cluster 11:
User User685 logged out.
 User User395 logged in.
 User User225 logged in.
User User494 logged out.
 User User900 logged in.

Cluster 13:
Backup started at 2025-05-14 07:06:55.
Backup started at 2025-02-15 20:00:19.
  Backup ended at 2025-08-08 13:06:23.
Backup started at 2025-11-14 08:27:43.
Backup started at 2025-12-09 10:19:11.

Cluster 7:
Multiple bad login attempts detected on user 85...
Multiple login failures occurred on user 9052 a...
  User 

In [64]:
# identifying most commonly appearing pattern through regex
import regex as re
def classify_with_regex(log_message) :
    regex_patterns = {
r"User User\d+ logged (in|out). ": "User Action",
r"Backup (started|ended) at .* ": "System Notification",
r"Backup completed successfully. ": "System Notification",
r"System updated to version .* ": "System Notification",
r"File . * uploaded successfully by user .* ": "System Notification",
r"Disk cleanup completed successfully. ": "System Notification",
r"System reboot initiated by user . * ": "System Notification",
r"Account with ID .* created by .* ": "User Action"
}  
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message,re.IGNORECASE):
            return label
    return None


In [65]:
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df.head()

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None


In [66]:
# new df without regex pattern
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,7,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,0,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,1,None


In [67]:
vc = df_non_regex['target_label'].value_counts()
print(vc[vc <= 5].index.tolist())


['Workflow Error', 'Deprecation Warning']


In [68]:
df_non_legacy = df_non_regex[df_non_regex.source!='LegacyCPRM']
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [69]:
# generate embeddings for log message
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())
filtered_embeddings[:2]

array([[-1.02939650e-01,  3.35459337e-02, -2.20260564e-02,
         1.55101169e-03, -9.86912474e-03, -1.78956285e-01,
        -6.34410679e-02, -6.01761974e-02,  2.81109493e-02,
         5.99619560e-02, -1.72618292e-02,  1.43372314e-03,
        -1.49560094e-01,  3.15284380e-03, -5.66030778e-02,
         2.71685515e-02, -1.49890240e-02, -3.54037099e-02,
        -3.62936929e-02, -1.45410625e-02, -5.61489025e-03,
         8.75538290e-02,  4.55121100e-02,  2.50964109e-02,
         1.00187529e-02,  1.24266976e-02, -1.39923558e-01,
         7.68696368e-02,  3.14095467e-02, -4.15253872e-03,
         4.36902903e-02,  1.71250310e-02, -8.00950900e-02,
         5.74006513e-02,  1.89091228e-02,  8.55261683e-02,
         3.96398529e-02, -1.34371877e-01, -1.44363148e-03,
         3.06704850e-03,  1.76854059e-01,  4.44884459e-03,
        -1.69274323e-02,  2.24267393e-02, -4.35050912e-02,
         6.09031925e-03, -9.98168997e-03, -6.23972267e-02,
         1.07371937e-02, -6.04898203e-03, -7.14661032e-0

In [ ]:
df_legacydf['source']